<a href="https://colab.research.google.com/github/OshadhaAK/ML_Competition/blob/master/final_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

In [ ]:
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import f1_score
import warnings
warnings.simplefilter('ignore')
import pandas.testing as tm
import seaborn as sns
sns.set()
RANDOM_SEED = 8


In [ ]:
df = pd.read_csv('/content/drive/My Drive/ml/train.csv',index_col='tripid')
Xtest = pd.read_csv('/content/drive/My Drive/ml/test.csv',index_col='tripid')
submission_df = pd.read_csv("/content/drive/My Drive/ml/sample_submission.csv",index_col='tripid')
t_dist = pd.read_csv("/content/drive/My Drive/ml/train_.csv")
x_dist = pd.read_csv("/content/drive/My Drive/ml/test_.csv")

In [ ]:
df

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,correct
213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,correct
213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,correct


In [ ]:
df['label'] = [0 if x == "incorrect" else 1 for x in df['label']]

In [ ]:
t_dist

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,distance,fare,label
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,7.801,270.32,correct
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,5.211,197.85,correct
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,9.637,301.64,correct
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,1.880,82.30,correct
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,9.998,358.39,correct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,3.943,198.26,correct
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,17.569,581.23,correct
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,2.697,76.20,correct
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,3.398,133.31,correct


In [ ]:
df['distance'] = t_dist['distance'].values
Xtest['distance'] = x_dist['distance'].values

In [ ]:
df

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label,distance
tripid,,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,1,7.801
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,1,5.211
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,1,9.637
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,1,1.880
189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,1,9.998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,1,3.943
213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,1,17.569
213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,1,2.697


In [ ]:
Xtest

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,distance
tripid,,,,,,,,,,,,,
213284604,10.5,924,42,2.44860,148,2/1/2020 0:38,2/1/2020 0:53,6.83454,79.8750,6.77490,79.8840,289.27,8.283
213286352,10.5,4249,20,0.00000,91,2/1/2020 1:02,2/1/2020 2:13,6.91168,79.8723,6.55091,79.9706,1912.70,56.384
213293973,10.5,1552,255,2.65880,23,2/1/2020 5:02,2/1/2020 5:28,6.92145,79.8478,6.90539,79.8989,394.00,7.760
213294622,10.5,462,16,0.00000,198,2/1/2020 5:30,2/1/2020 5:38,6.77433,79.9416,6.80401,79.9407,154.32,5.015
213298687,10.5,814,392,12.36920,69,2/1/2020 7:00,2/1/2020 7:14,6.97968,79.9130,6.98875,79.8914,147.47,3.046
...,...,...,...,...,...,...,...,...,...,...,...,...,...
222856243,10.5,1723,429,24.83332,3,3/16/2020 21:28,3/16/2020 21:56,6.85103,79.9567,6.85588,79.9214,388.48,5.853
222857785,10.5,1378,80,0.00000,125,3/16/2020 21:59,3/16/2020 22:22,6.91293,79.9656,6.92112,79.8980,379.85,10.815
222858416,10.5,418,56,3.28440,93,3/16/2020 22:02,3/16/2020 22:09,6.85718,79.9081,6.83868,79.9083,112.79,2.300


In [ ]:
df = df.dropna()

In [ ]:
df.isna().sum()

additional_fare              0
duration                     0
meter_waiting                0
meter_waiting_fare           0
meter_waiting_till_pickup    0
pickup_time                  0
drop_time                    0
pick_lat                     0
pick_lon                     0
drop_lat                     0
drop_lon                     0
fare                         0
label                        0
distance                     0
dtype: int64

In [ ]:
df

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label,distance
tripid,,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,1,7.801
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,1,5.211
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,1,9.637
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,1,1.880
189129552,10.5,3407.0,182.0,0.0000,112.0,11/1/2019 5:38,11/1/2019 6:35,7.13402,79.8969,6.91865,79.8649,1065.02,1,31.163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,1,3.943
213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,1,17.569
213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,1,2.697


In [ ]:
df['cost'] = [d >= 300 for d in df['fare']]
df['cost'] = df['cost'].map({True: 1, False: 0})

Xtest['cost'] = [d >= 300 for d in Xtest['fare']]
Xtest['cost'] = Xtest['cost'].map({True: 1, False: 0})

In [ ]:
df['fare_min'] = df['fare']/df['distance']
Xtest['fare_min'] = Xtest['fare']/Xtest['distance']

In [ ]:
df['additional_fare_min'] = df['additional_fare']/df['distance']
Xtest['additional_fare_min'] = Xtest['additional_fare']/Xtest['distance']

In [ ]:
df['diff_lon'] = df['pick_lon'] - df['drop_lon']
df['diff_lat'] = df['pick_lat'] - df['drop_lat']

Xtest['diff_lon'] = Xtest['pick_lon'] - Xtest['drop_lon']
Xtest['diff_lat'] = Xtest['pick_lat'] - Xtest['drop_lat']

In [ ]:
df['pickup_time'] = pd.to_datetime(df['pickup_time'], errors='coerce')
df['drop_time'] = pd.to_datetime(df['drop_time'], errors='coerce')
df['pickup_time_hour'] = df['pickup_time'].dt.hour
df['drop_time_hour'] = df['drop_time'].dt.hour
df['pickup_time_minute'] = df['pickup_time'].dt.minute
df['drop_time_minute'] = df['drop_time'].dt.minute
# df['pickup_time_dayofweek'] = df['pickup_time'].dt.dayofweek.astype('float')
# df['drop_time_dayofweek'] = df['drop_time'].dt.dayofweek.astype('float')
# df['pickup_time_day'] = df['pickup_time'].dt.day

Xtest['pickup_time'] = pd.to_datetime(Xtest['pickup_time'], errors='coerce')
Xtest['drop_time'] = pd.to_datetime(Xtest['drop_time'], errors='coerce')
Xtest['pickup_time_hour'] = Xtest['pickup_time'].dt.hour
Xtest['drop_time_hour'] = Xtest['drop_time'].dt.hour
Xtest['pickup_time_minute'] = Xtest['pickup_time'].dt.minute
Xtest['drop_time_minute'] = Xtest['drop_time'].dt.minute
# Xtest['pickup_time_dayofweek'] = Xtest['pickup_time'].dt.dayofweek.astype('float')
# Xtest['drop_time_dayofweek'] = Xtest['drop_time'].dt.dayofweek.astype('float')
# Xtest['pickup_time_day'] = Xtest['pickup_time'].dt.day

In [ ]:
def calculate_direction(d_lon, d_lat):
    result = np.zeros(len(d_lon))
    l = np.sqrt(d_lon**2 + d_lat**2)
    result[d_lon>0] = (180/np.pi)*np.arcsin(d_lat[d_lon>0]/l[d_lon>0])
    idx = (d_lon<0) & (d_lat>0)
    result[idx] = 180 - (180/np.pi)*np.arcsin(d_lat[idx]/l[idx])
    idx = (d_lon<0) & (d_lat<0)
    result[idx] = -180 - (180/np.pi)*np.arcsin(d_lat[idx]/l[idx])
    return result

In [ ]:
df['direction'] = calculate_direction(df.diff_lon, df.diff_lat)
Xtest['direction'] = calculate_direction(Xtest.diff_lon, Xtest.diff_lat)

In [ ]:
df['riding_fare_ratio'] = (df['fare']/abs(df['duration'] +1 - df['meter_waiting']))*60
Xtest['riding_fare_ratio'] = (Xtest['fare']/abs(Xtest['duration'] +1 - Xtest['meter_waiting']))*60

In [ ]:
df['speed'] = df['distance']/(df['duration']+1)
Xtest['speed'] = Xtest['distance']/(Xtest['duration']+1)

In [ ]:
df = df.drop(['drop_time','pickup_time','diff_lon','diff_lat'], axis = 1)
Xtest = Xtest.drop(['drop_time','pickup_time','diff_lon','diff_lat'], axis = 1)

In [ ]:
np.testing.assert_array_equal(Xtest.index.values, submission_df.index.values)

In [ ]:
X = df.drop('label',1)
y = df.label

In [ ]:
X

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,distance,cost,fare_min,additional_fare_min,pickup_time_hour,drop_time_hour,pickup_time_minute,drop_time_minute,direction,riding_fare_ratio,speed
tripid,,,,,,,,,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,7.801,0,34.651968,1.345981,0,0,20,34,-62.753420,20.820539,0.009343
189125358,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,5.211,0,37.967761,2.014968,0,1,56,9,-77.641274,15.934228,0.006580
189125719,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,9.637,1,31.300197,1.089551,1,1,8,26,-150.242655,17.954762,0.008858
189127273,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,1.880,0,43.776596,5.585106,2,2,27,37,-166.818323,15.054878,0.003139
189129552,10.5,3407.0,182.0,0.0000,112.0,7.13402,79.8969,6.91865,79.8649,1065.02,31.163,1,34.175785,0.336938,5,6,38,35,81.548736,19.808184,0.009144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213803193,10.5,838.0,93.0,5.4219,451.0,7.29073,80.6367,7.28891,80.6557,198.26,3.943,0,50.281512,2.662947,22,22,7,21,174.528362,15.945845,0.004700
213812756,10.5,2151.0,428.0,0.0000,39.0,6.90569,79.8516,6.95089,79.9389,581.23,17.569,1,33.082702,0.597644,23,23,7,43,-152.626918,20.228422,0.008164
213813930,10.5,263.0,9.0,0.0000,110.0,7.09210,79.9000,7.10135,79.9017,76.20,2.697,0,28.253615,3.893215,23,23,21,25,-100.413825,17.929412,0.010216


In [ ]:
print (y.value_counts())

1    15442
0     1526
Name: label, dtype: int64


In [ ]:


model = CatBoostClassifier(learning_rate=0.02)


model.fit(X,y)

0:	learn: 0.6618490	total: 15.6ms	remaining: 15.5s
1:	learn: 0.6360632	total: 29.2ms	remaining: 14.6s
2:	learn: 0.6079637	total: 42.7ms	remaining: 14.2s
3:	learn: 0.5815552	total: 55.6ms	remaining: 13.9s
4:	learn: 0.5566866	total: 68.3ms	remaining: 13.6s
5:	learn: 0.5344345	total: 81.5ms	remaining: 13.5s
6:	learn: 0.5121016	total: 97.2ms	remaining: 13.8s
7:	learn: 0.4913001	total: 112ms	remaining: 13.9s
8:	learn: 0.4728559	total: 125ms	remaining: 13.8s
9:	learn: 0.4561030	total: 138ms	remaining: 13.7s
10:	learn: 0.4402202	total: 155ms	remaining: 14s
11:	learn: 0.4241526	total: 168ms	remaining: 13.8s
12:	learn: 0.4083521	total: 182ms	remaining: 13.8s
13:	learn: 0.3937357	total: 196ms	remaining: 13.8s
14:	learn: 0.3808660	total: 209ms	remaining: 13.7s
15:	learn: 0.3686616	total: 226ms	remaining: 13.9s
16:	learn: 0.3573481	total: 239ms	remaining: 13.8s
17:	learn: 0.3467019	total: 251ms	remaining: 13.7s
18:	learn: 0.3365138	total: 265ms	remaining: 13.7s
19:	learn: 0.3283153	total: 284ms	re

In [ ]:
ynew = model.predict(Xtest)


In [ ]:
ynew

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
import collections
print (collections.Counter(ynew))


Counter({1: 8148, 0: 428})


In [ ]:
res = pd.DataFrame({'tripid': Xtest.index, 'prediction': ynew})
res.to_csv("/content/drive/My Drive/ml/test62.csv", index=False)

In [ ]:
!head '/content/drive/My Drive/ml/test62.csv'

tripid,prediction
213284604,1
213286352,1
213293973,1
213294622,1
213298687,1
213299545,0
213302332,1
213302671,1
213305594,1
